# Extract Environmental Features

**Timm Nawrocki**  
Alaska Center for Conservation Science  
2019-04-09

In [1]:
# -*- coding: utf-8 -*-
# ---------------------------------------------------------------------------
# Extract Environmental Features
# Author: Timm Nawrocki
# Created on: 2019-04-09
# Usage: Must be executed as a Jupyter Notebook in an ArcGIS Pro Python 3 installation.
# Description: "Extract Environmental Features" subsets the environmental features for the modeling area to a smaller mask raster.

## 1. Initialize Environment

In [2]:
# Import packages
import arcpy
from arcpy.sa import *
import datetime
import os
import time

# Set root directory
drive = 'F:/'
root_directory = os.path.join(drive, 'ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS')

# Set arcpy working environment
arcpy.env.workspace = os.path.join(root_directory, 'BristolBay_Vegetation.gdb')

# Define script paths
script_directory = os.path.join(drive, 'ACCS_Work/Repositories/southwest-alaska-moose/modules')
arcpy_geoprocessing_script = os.path.join(script_directory, 'arcpy_geoprocessing.py')

# Define input data
mask_raster = os.path.join(root_directory, 'Data_Input/area_of_interest/NuyakukAleknagik_Mask.tif')
features_full = os.path.join(root_directory, 'Data_Input/predictor_env/full')

# Define output directories
features_partial = os.path.join(root_directory, 'Data_Input/predictor_env/partial')
        
# List the environmental features
environmental_list = []
for file in os.listdir(features_full):
    if file.endswith('.tif'):
        environmental_list.append(file)

In [3]:
# Import and execute arcpy_geoprocessing.py
try:
    exec(open(arcpy_geoprocessing_script).read())
except:
    print("Error loading arcpy_geoprocessing.py; ensure that script directory is correct:")
    print(script_directory)
    quit()

## 2. Extract raster to mask

In [4]:
# Define a function to extract environmental features
def extract_raster(**kwargs):
    """
    Description: extracts environmental features to a mask raster
    Inputs: 'input_array' -- an array containing the input rasters with the first raster as the mask raster and the second as the input raster
            'output_array' -- an array of one output name
    """
    # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    mask_raster = kwargs['input_array'][0]
    input_raster = kwargs['input_array'][1]
    output_raster = kwargs['output_array'][0]
    # If output does not exist then process rasters
    if arcpy.Exists(output_raster) == False:
        print('----------')
        print('Processing raster tile {0}...'.format(os.path.split(output_raster)[1]))
        # Set the snap raster and cell size environments to match the area of interest
        arcpy.env.snapRaster = mask_raster
        arcpy.env.cellSize = mask_raster
        # Extract filled raster to mask raster
        print('Extracting raster to mask...')
        extract_raster = ExtractByMask(input_raster, mask_raster)
        # Copy extract raster to output
        print('Saving output...')
        arcpy.CopyRaster_management(extract_raster,
                                    output_raster,
                                    '',
                                    '',
                                    '-32768',
                                    'NONE',
                                    'NONE',
                                    '32_BIT_FLOAT',
                                    'NONE',
                                    'NONE',
                                    'TIFF',
                                    'NONE'
                                   )
        # End timing function execution and calculate elapsed time
        end = time.time()
        elapsed = int(end - start)
        success_time = datetime.datetime.now()
        # Report process success
        print('Successfully extracted environmental feature...')
        print('----------')
        out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                    datetime.timedelta(seconds=elapsed))
        return out_process

In [5]:
# Run the mosaic and interpolation process for each tile set in list
for raster in environmental_list:
    # Define input and output raster
    input_raster = os.path.join(features_full, raster)
    output_raster = os.path.join(features_partial, raster)
    # Define input and output arrays
    extract_raster_inputs = [mask_raster, input_raster]
    extract_raster_outputs = [output_raster]
    # Create key word arguments
    extract_raster_kwargs = {'input_array' : extract_raster_inputs,
                             'output_array' : extract_raster_outputs
                            }
    # Mosaic and interpolate image
    arcpy_geoprocessing(extract_raster, check_output = False, **extract_raster_kwargs)

----------
Processing raster tile aspect.tif...
Extracting raster to mask...
Saving output...
Successfully extracted environmental feature...
----------
Succeeded at 2019-04-09 18:58 (Elapsed time: 0:00:56)
----------
Processing raster tile compoundTopographic.tif...
Extracting raster to mask...
Saving output...
Successfully extracted environmental feature...
----------
Succeeded at 2019-04-09 18:59 (Elapsed time: 0:00:56)
----------
Processing raster tile dateFreeze.tif...
Extracting raster to mask...
Saving output...
Successfully extracted environmental feature...
----------
Succeeded at 2019-04-09 19:00 (Elapsed time: 0:00:33)
----------
Processing raster tile dateThaw.tif...
Extracting raster to mask...
Saving output...
Successfully extracted environmental feature...
----------
Succeeded at 2019-04-09 19:00 (Elapsed time: 0:00:32)
----------
Processing raster tile elevation.tif...
Extracting raster to mask...
Saving output...
Successfully extracted environmental feature...
--------